In [1]:
import torch.optim as optim
import torch.nn as nn
from qiskit.providers.fake_provider import FakeLima
from scripts.data_setup import load_data
from scripts.from_circ_to_numpy import operations_to_features, save_to_json, load_from_json
from scripts.model import create_models, train_and_test_step

In [2]:
n_qubits = 5 # 5 == n_qubits, default value for now because of FakeLima
train_circuits, train_observables, train_ideal_exp_vals, train_noisy_exp_vals, test_circuits, test_observables, test_ideal_exp_vals, test_noisy_exp_vals = load_data('data/circuits/data_test_1')

In [3]:
X_train, y_train, X_test, y_test = load_from_json('data/features/data_test_1')

In [4]:
# specify these :)
sequence_input_size = 7
sequence_hidden_size = 7
sequence_num_layers = 2
sequence_model_type = 'Transformer'
sequence_dropout = 0.1
ann_hidden_layers = 4
ann_hidden_units = 64
ann_dropout = 0.1
noisy_first = True

sequence_model, ann = create_models(sequence_input_size, 
                                    sequence_hidden_size, 
                                    sequence_num_layers, 
                                    sequence_model_type, 
                                    sequence_dropout, 
                                    ann_hidden_layers, 
                                    ann_hidden_units, 
                                    ann_dropout, 
                                    noisy_first=noisy_first)

sequence_config = {
    "input_size": sequence_input_size,
    "hidden_size": sequence_hidden_size,
    "num_layers": sequence_num_layers,
    "model_type": sequence_model_type,
    "dropout": sequence_dropout
}

ann_config = {
    "hidden_layers": ann_hidden_layers,
    "hidden_units": ann_hidden_units,
    "dropout": ann_dropout,
    "noisy_first": noisy_first
}

c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [5]:
loss_fn = nn.MSELoss() # leave for now
optimizer = optim.Adam(list(ann.parameters()) + list(sequence_model.parameters()), lr=0.001)

num_epochs = 5
train_and_test_step(sequence_model, ann, loss_fn, optimizer, X_train, train_noisy_exp_vals, y_train, X_test, test_noisy_exp_vals, y_test, num_epochs, noisy_first=noisy_first)

  0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
optimizer = optim.Adam(list(ann.parameters()) + list(sequence_model.parameters()), lr=0.0001)
train_and_test_step(sequence_model, ann, loss_fn, optimizer, X_train, train_noisy_exp_vals, y_train, X_test, test_noisy_exp_vals, y_test, num_epochs, noisy_first=noisy_first)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5, train loss: 0.2857, test_loss: 0.2503
Epoch 2/5, train loss: 0.2860, test_loss: 0.2427
Epoch 3/5, train loss: 0.2791, test_loss: 0.2469
Epoch 4/5, train loss: 0.2753, test_loss: 0.2522
Epoch 5/5, train loss: 0.2757, test_loss: 0.2610
